In [ ]:
%pip install requests pandas

In [30]:
relevant_parties = ["CDU", "SPD", "GRÜNE", "LINKE", "AfD", "FDP"]

In [ ]:
import requests;
import pandas as pd
from collections import defaultdict

base_url = "http://r8swgk4cgokosk08kc00swsg.82.165.43.153.sslip.io/"
elections = requests.get(base_url + "election").json()
results = []
for election in elections:
    result = requests.get(base_url + "election/" + str(election["id"]) + "?primary_vote=false").json()
    if election["name"] not in ["Bundestagswahl", "Europawahl"]:
        continue    
    voters = 0
    parties = defaultdict(int)
    
    for region in result["region"]: 
        voters += region["turnout"][0]["voted"]
        for vote in region["votes"]:
            if vote["id"] in relevant_parties:
                idx = relevant_parties.index(vote["id"])
                parties[relevant_parties[idx]] += vote["votes"]
    for (key, value) in parties.items():
        parties[key] = value / voters
    result = {"name": election["name"] + str(election["date"]), **parties}
    results.append(result)
    
bs = pd.DataFrame(results)

bund = pd.read_csv("db/results_germany.csv", sep=";")